Imports

In [4]:
import io
import os
import re
import logging

import pdfplumber
import pytesseract
from pdf2image import convert_from_bytes

import pandas as pd
from sqlalchemy import create_engine
from s3_manager import S3Manager  # adjust path if needed

import fitz                # PyMuPDF
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed


Get Tickers

In [3]:
# === 1) TICKER FETCHER ===

def get_all_tickers():
    url = "https://www.asx.com.au/asx/research/ASXListedCompanies.csv"
    df = pd.read_csv(url, skiprows=1)

    # detect sector column
    possible = ["Industry Group", "GICS industry group", "Industry", "GICS Sector"]
    col = next((c for c in possible if c in df.columns), None)
    df["Sector"] = df[col].str.strip() if col else "Unknown"

    df["Ticker"] = df["ASX code"].str.upper().str.strip()
    return df[df["Sector"] == "Capital Goods"][["Ticker"]]

PDF Extraction

In [8]:
def clean_holder_name(raw_name: str) -> str:
    # your existing cleaning logic here
    return raw_name.strip()

def extract_from_pdf_bytes_with_ocr(pdf_bytes: bytes) -> dict:
    """
    1) Try pdfplumber for text extraction
    2) Fall back to PyMuPDF's get_text()
    3) Fall back to OCR via pytesseract
    Returns a dict with method, holder, number_of_securities, persons_votes, voting_power.
    """
    vote_pattern = re.compile(
        r"Class of securities.*?Ordinary\s+([\d,]+)\s+([\d,]+)\s+([\d.]+)%",
        re.DOTALL | re.IGNORECASE
    )
    name_pattern = re.compile(r"(?i)Name\s+(.+)")
    
    # 1) pdfplumber
    with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            # extract holder
            m_name = name_pattern.search(txt)
            holder = clean_holder_name(m_name.group(1)) if m_name else None
            # extract votes
            m_votes = vote_pattern.search(txt)
            if m_votes:
                return {
                    "method": "pdfplumber",
                    "holder": holder,
                    "number_of_securities": int(m_votes.group(1).replace(",", "")),
                    "persons_votes":        int(m_votes.group(2).replace(",", "")),
                    "voting_power":         m_votes.group(3) + "%",
                }

    # 2) PyMuPDF fallback
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    for page in doc:
        txt = page.get_text() or ""
        m_name = name_pattern.search(txt)
        holder = clean_holder_name(m_name.group(1)) if m_name else None
        m_votes = vote_pattern.search(txt)
        if m_votes:
            return {
                "method": "pymupdf",
                "holder": holder,
                "number_of_securities": int(m_votes.group(1).replace(",", "")),
                "persons_votes":        int(m_votes.group(2).replace(",", "")),
                "voting_power":         m_votes.group(3) + "%",
            }
    
    # 3) OCR fallback
    # Render each page to an image and OCR it
    for page_index in range(len(doc)):
        page = doc[page_index]
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        txt = pytesseract.image_to_string(img)
        
        m_name = name_pattern.search(txt)
        holder = clean_holder_name(m_name.group(1)) if m_name else None
        m_votes = vote_pattern.search(txt)
        if m_votes:
            return {
                "method": "ocr",
                "holder": holder,
                "number_of_securities": int(m_votes.group(1).replace(",", "")),
                "persons_votes":        int(m_votes.group(2).replace(",", "")),
                "voting_power":         m_votes.group(3) + "%",
            }
    
    # nothing found
    return {
        "method": None,
        "holder": None,
        "number_of_securities": None,
        "persons_votes": None,
        "voting_power": None
    }

Main

In [10]:
# === 3) MAIN WORKFLOW ===
def fetch_and_extract(fname, s3m, bucket):
    """
    Download the PDF for fname, run extract_from_pdf_bytes,
    and return a dict with Filename + extraction fields.
    """
    date_folder, file_id = fname[:8], fname[8:]
    key = f"{date_folder}/{file_id}.pdf"

    try:
        resp      = s3m.s3.get_object(Bucket=bucket, Key=key)
        pdf_bytes = resp["Body"].read()
    except Exception as e:
        print(f"⚠️ Failed to fetch {key}: {e}")
        return None

    data = extract_from_pdf_bytes_with_ocr(pdf_bytes)
    data["Filename"] = fname
    return data

def main():
    # 1) Load Capital Goods tickers
    tickers = get_all_tickers()["Ticker"].tolist()

    # 2) Connect to your RDS MySQL
    engine = create_engine(
        "mysql+pymysql://sam:sam2025@"
        "database-1.cmy0wo2batmu.ap-southeast-2.rds.amazonaws.com:3306/ASX_Market"
    )

    # 3) Pull ASX_RepTypes, filter by your codes + tickers
    rep_df = pd.read_sql_table("ASX_RepTypes", engine, schema="ASX_Market")
    codes = ["02001"]  # add more as needed
    mask = rep_df["RepType Code"].isin(codes) & rep_df["Ticker"].isin(tickers)
    files = rep_df.loc[mask, "Filename"].tolist()
    print(f"✅ Found {len(files)} matching files\n")

   # 5) Stream each PDF, extract in parallel
    s3m    = S3Manager()
    bucket = "xtf-asx"
    records = []

    # choose number of worker threads (e.g. 8)
    max_workers = 25
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # schedule all tasks
        future_to_fname = {
            executor.submit(fetch_and_extract, fname, s3m, bucket): fname
            for fname in files
        }
        # as each finishes, collect results
        for future in as_completed(future_to_fname):
            res = future.result()
            if res:
                records.append(res)

    # 6) Assemble & save as before…
    df_out = pd.DataFrame(records, columns=[
        "Filename", "method", "holder",
        "previous_votes", "previous_pct",
        "present_votes", "present_pct"
    ])
    out_path = r"C:\Users\HarryBox\Documents\SK_Investair\ASX_SS_Forms\FormsProject\output.csv"
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved output to {out_path}\n")
    print(df_out.to_markdown(index=False))


if __name__ == "__main__":
    main()

✅ Found 150 matching files



CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.